In [1]:
import ddm_data_simulation as ddm_sim
import scipy as scp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
#import kde_class
import multiprocessing as mp
import psutil
import pickle 
import os
import re

import kde_training_utilities as kde_util
import kde_class as kde

In [ ]:
kde_util.kde_train_test_from_simulations_flexbound(base_simulation_folder = os.getcwd() + '/data_storage/kde/base_simulations/',
                                         target_folder = os.getcwd() + '/data_storage/kde/kde_ddm_flexbound_train_test_extended_params/',
                                         n_total = 200000000,
                                         p_train = 0.9,
                                         mixture_p = [0.8, 0.1, 0.1],
                                         model = 'ddm_flexbound',
                                         print_info = False,
                                         target_file_format = 'pickle',
                                         n_files_max = 'all')

1470
2940
no_base_data reported
4410
[[0.73]]
no_base_data reported
5880
7350
8820
10290
11760
13230
14700
16170
17640
19110
20580
22050
[[3.9]]
no_base_data reported
23520
24990
no_base_data reported
26460
27930
29400
no_base_data reported
30870
no_base_data reported
32340
33810
35280
36750
38220
39690
41160
42630
no_base_data reported
44100
45570
47040
48510
49980
[[0.7]]
no_base_data reported
51450
52920
no_base_data reported
54390
55860
57330
58800
no_base_data reported
60270
61740
63210
64680
66150
67620
69090
70560
72030
73500
74970
[[1.34]]
no_base_data reported
76440
77910
no_base_data reported
79380
80850
82320
83790
85260
86730
88200
89670
91140
92610
94080
95550
97020
98490
99960
101430
102900
104370
105840
107310
108780
110250
no_base_data reported
111720
113190
114660
116130
117600
119070
120540
122010
123480
[[4.6]]
no_base_data reported
124950
126420
127890
129360
130830
132300
133770
135240
136710
138180
139650
141120
142590
144060
145530
147000
148470
149940
no_base_da

1198050
1199520
no_base_data reported
1200990
1202460
1203930
[[2.52]]
no_base_data reported
1205400
1206870
1208340
1209810
1211280
1212750
1214220
1215690
1217160
1218630
1220100
1221570
1223040
1224510
1225980
1227450
[[0.72]]
no_base_data reported
1228920
1230390
1231860
1233330
1234800
1236270
1237740
1239210
1240680
1242150
1243620
1245090
1246560
1248030
1249500
1250970
no_base_data reported
1252440
1253910
1255380
[[0.65]]
no_base_data reported
1256850
1258320
1259790
1261260
1262730
1264200
1265670
1267140
1268610
1270080
1271550
[[2.37]]
no_base_data reported
1273020
1274490
1275960
no_base_data reported
1277430
1278900
[[4.1]]
no_base_data reported
1280370
1281840
1283310
1284780
1286250
1287720
1289190
1290660
1292130
no_base_data reported
1293600
1295070
1296540
1298010
no_base_data reported
1299480
1300950
1302420
1303890
1305360
1306830
1308300
1309770
1311240
no_base_data reported
1312710
1314180
1315650
1317120
1318590
1320060
1321530
1323000
1324470
no_base_data repor

2329950
2331420
[[1.15]]
no_base_data reported
2332890
2334360
2335830
2337300
2338770
no_base_data reported
2340240
no_base_data reported
2341710
2343180
2344650
2346120
2347590
2349060
2350530
2352000
2353470
2354940
no_base_data reported
2356410
no_base_data reported
2357880
2359350
2360820
2362290
2363760
[[2.89]]
no_base_data reported
2365230
2366700
2368170
no_base_data reported
2369640
2371110
2372580
2374050
2375520
2376990
2378460
2379930
2381400
2382870
2384340
2385810
2387280
2388750
2390220
no_base_data reported
2391690
2393160
no_base_data reported
2394630
2396100
2397570
no_base_data reported
2399040
2400510
2401980
2403450
2404920
2406390
2407860
2409330
2410800
2412270
2413740
2415210
2416680
[[1.43]]
no_base_data reported
2418150
2419620
2421090
2422560
2424030
2425500
2426970
2428440
2429910
2431380
2432850
2434320
2435790
2437260
2438730
2440200
2441670
2443140
[[1.11]]
no_base_data reported
2444610
2446080
2447550
2449020
2450490
2451960
2453430
2454900
2456370
2457

3442740
3444210
3445680
[[3.06]]
no_base_data reported
3447150
3448620
3450090
3451560
3453030
3454500
3455970
3457440
no_base_data reported
3458910
3460380
3461850
3463320
3464790
no_base_data reported
3466260
3467730
3469200
3470670
3472140
no_base_data reported
3473610
3475080
3476550
3478020
3479490
no_base_data reported
3480960
3482430
3483900
3485370
3486840
3488310
3489780
3491250
3492720
3494190
3495660
3497130
3498600
3500070
3501540
3503010
3504480
3505950
3507420
no_base_data reported
3508890
3510360
3511830
3513300
3514770
3516240
3517710
3519180
3520650
no_base_data reported
3522120
3523590
3525060
3526530
[[0.67]]
no_base_data reported
3528000
3529470
3530940
3532410
3533880
3535350
3536820
3538290
3539760
3541230
3542700
3544170
3545640
3547110
3548580
3550050
3551520
no_base_data reported
3552990
3554460
3555930
3557400
3558870
3560340
3561810
3563280
3564750
3566220
3567690
3569160
no_base_data reported
3570630
3572100
no_base_data reported
3573570
3575040
[[0.59]]
no_

4543770
4545240
4546710
4548180
4549650
no_base_data reported
4551120
4552590
4554060
4555530
4557000
4558470
4559940
4561410
4562880
4564350
4565820
4567290
4568760
4570230
4571700
4573170
4574640
4576110
4577580
no_base_data reported
4579050
4580520
4581990
4583460
4584930
4586400
4587870
4589340
4590810
4592280
4593750
4595220
4596690
4598160
4599630
4601100
4602570
[[3.09]]
no_base_data reported
4604040
4605510
4606980
4608450
no_base_data reported
4609920
4611390
4612860
no_base_data reported
4614330
4615800
4617270
4618740
4620210
4621680
4623150
4624620
4626090
4627560
4629030
4630500
4631970
4633440
4634910
[[1.08]]
no_base_data reported
4636380
4637850
4639320
no_base_data reported
4640790
4642260
4643730
4645200
4646670
4648140
no_base_data reported
4649610
4651080
4652550
4654020
4655490
4656960
4658430
4659900
no_base_data reported
4661370
4662840
4664310
4665780
4667250
4668720
4670190
4671660
4673130
4674600
no_base_data reported
4676070
4677540
4679010
4680480
4681950
no

5710950
[[1.87]]
no_base_data reported
5712420
5713890
5715360
5716830
5718300
5719770
5721240
5722710
5724180
[[1.74]]
no_base_data reported
5725650
5727120
5728590
5730060
5731530
5733000
5734470
5735940
5737410
no_base_data reported
5738880
5740350
5741820
[[1.9]]
no_base_data reported
5743290
5744760
5746230
5747700
5749170
5750640
5752110
5753580
5755050
5756520
5757990
5759460
5760930
5762400
5763870
5765340
5766810
5768280
5769750
5771220
5772690
5774160
5775630
5777100
5778570
5780040
5781510
[[0.91]]
no_base_data reported
5782980
5784450
5785920
5787390
5788860
5790330
5791800
no_base_data reported
5793270
5794740
5796210
5797680
5799150
5800620
5802090
5803560
5805030
5806500
5807970
5809440
5810910
5812380
5813850
5815320
5816790
no_base_data reported
5818260
5819730
5821200
5822670
5824140
5825610
5827080
5828550
5830020
5831490
5832960
5834430
5835900
5837370
no_base_data reported
5838840
5840310
5841780
5843250
5844720
5846190
5847660
5849130
5850600
5852070
no_base_data 

6847260
6848730
6850200
6851670
6853140
6854610
6856080
6857550
6859020
6860490
6861960
6863430
6864900
[[0.35]]
no_base_data reported
6866370
6867840
no_base_data reported
6869310
6870780
6872250
6873720
no_base_data reported
6875190
6876660
6878130
6879600
6881070
6882540
6884010
6885480
6886950
6888420
6889890
6891360
no_base_data reported
6892830
6894300
6895770
6897240
6898710
6900180
6901650
no_base_data reported
6903120
6904590
6906060
6907530
6909000
6910470
6911940
6913410
6914880
6916350
6917820
6919290
6920760
6922230
6923700
6925170
6926640
6928110
6929580
6931050
6932520
6933990
6935460
no_base_data reported
6936930
6938400
6939870
6941340
no_base_data reported
6942810
6944280
6945750
6947220
6948690
6950160
6951630
6953100
6954570
6956040
6957510
6958980
6960450
6961920
6963390
6964860
6966330
6967800
no_base_data reported
6969270
6970740
6972210
6973680
6975150
6976620
6978090
6979560
6981030
6982500
6983970
6985440
6986910
6988380
6989850
6991320
6992790
6994260
no_base

8017380
8018850
8020320
8021790
8023260
8024730
8026200
8027670
8029140
8030610
8032080
8033550
8035020
8036490
8037960
8039430
no_base_data reported
8040900
8042370
8043840
8045310
8046780
8048250
8049720
8051190
8052660
8054130
8055600
8057070
[[3.79]]
no_base_data reported
8058540
8060010
8061480
8062950
8064420
8065890
8067360
8068830
8070300
8071770
no_base_data reported
8073240
8074710
no_base_data reported
8076180
8077650
8079120
8080590
8082060
8083530
8085000
8086470
no_base_data reported
8087940
8089410
8090880
8092350
8093820
8095290
8096760
8098230
no_base_data reported
8099700
8101170
8102640
8104110
8105580
8107050
8108520
no_base_data reported
8109990
8111460
8112930
8114400
8115870
8117340
8118810
8120280
8121750
no_base_data reported
8123220
8124690
[[1.64]]
no_base_data reported
8126160
8127630
8129100
8130570
8132040
8133510
8134980
no_base_data reported
8136450
8137920
8139390
no_base_data reported
8140860
8142330
8143800
8145270
8146740
8148210
8149680
8151150
no_b

9152220
9153690
9155160
9156630
9158100
9159570
9161040
9162510
no_base_data reported
9163980
9165450
9166920
9168390
9169860
no_base_data reported
9171330
9172800
9174270
9175740
9177210
9178680
9180150
no_base_data reported
9181620
9183090
9184560
9186030
9187500
9188970
9190440
9191910
9193380
9194850
9196320
9197790
[[3.61]]
no_base_data reported
9199260
9200730
no_base_data reported
9202200
9203670
no_base_data reported
9205140
9206610
9208080
9209550
no_base_data reported
9211020


In [ ]:
# Load some file and extract column names to be kept going forward
test = pickle.load(open(os.getcwd() + '/data_storage/kde/flexbound/kde_base_dat/kde_ddm_flexbound_fff84ea0681f11e98948897ab099f7d4.pickle', 'rb'))
col_names = list(test)
test_frame = pd.DataFrame(columns = ['cols_to_keep', 'kde_type', 'bandwidth_rule', 'mixture'])
test_frame.loc[0] = [col_names[:8], test['kde_type'][0], test['bandwidth_rule'][0], test['mixture'][0]]

In [ ]:
# Write meta file that stores the columns we are interested in in a csv
test_frame.to_csv(os.getcwd() + '/data_storage/kde/kde_base_dat/meta_data.csv', index = False)

In [ ]:
# Initializations
#meta = pd.read_csv(os.getcwd() + '/data_storage/kde/kde_base_dat/meta_data.csv')
simulator = 'kde_ddm_flexbound'
#to_folder = os.getcwd() + '/data_storage/kde/kde_base_dat/' # + simulator + '_' + 'test_full' 
test_folder = os.getcwd() + '/data_storage/kde/flexbound/' + simulator + '_' + 'reduced_files' 
final_folder = os.getcwd() + '/data_storage/kde/flexbound/' + simulator + '_' + 'final_train_test'
#file_list = os.listdir(to_folder)

In [ ]:
# Make reduced data (dropping unnecessary column w.r.t network training)
cnt = 0
for file_ in file_list:
    if file_ != 'meta_data.csv':
        my_mat = pickle.load(open(to_folder + '/' + file_, 'rb'))
        my_mat = my_mat[eval(meta['cols_to_keep'][0])]
        pickle.dump(my_mat, open(test_folder + '/' + file_, "wb"))
        cnt += 1
    print(cnt)

In [ ]:
# Make train test split now
file_list = os.listdir(test_folder)
file_list_with_dir = [test_folder + '/' + file for file in file_list]
df = pd.concat([pd.read_pickle(fp) for fp in file_list_with_dir], ignore_index = True)

In [ ]:
# Get training and test labels
train_id = np.random.choice(a = [True, False], 
                            size = df.shape[0], 
                            replace = True, 
                            p = [0.8, 0.2])
test_id  = np.invert(train_id)

In [ ]:
# Store training and test data to file 
df.iloc[train_id, :7].to_pickle(final_folder + '/train_features.pickle' , protocol = 4)
df.iloc[test_id, :7].to_pickle(final_folder + '/test_features.pickle', protocol = 4)
df.iloc[train_id, 7].to_pickle(final_folder + '/train_labels.pickle', protocol = 4)
df.iloc[test_id, 7].to_pickle(final_folder + '/test_labels.pickle', protocol = 4)

In [ ]:
df.iloc[0,:7]